In [1]:
from binance import Client

import numpy as np
import pandas as pd
import ta 
import time

In [2]:
api_key = 's20c6FhOSUORTSgkjROWRCekmWoyMtMnV3BEbistCKvX3OUckd4vkL1WCNF6iZ5x'
secret_key = 'vBmwXWkWTM5hfLbury8sMAfgI6Zfnw3ZLkq524Cve0KHwUpWNdeYcz21VdzTW69E'

client = Client(api_key, secret_key)

In [3]:
pairs = []
exchange_info = client.get_exchange_info()
for s in exchange_info['symbols']:
     if (s['symbol'][-4:] == 'USDT') & (s['status'] == 'TRADING') & ((s['symbol'][-6:-4] != 'UP') & (s['symbol'][-8:-4] != 'DOWN')):
        pairs.append(s['symbol'])

In [4]:
stream = pd.DataFrame(columns=['Name', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume'])

In [5]:
def get_data(symbol, interval, lookback):
    df = pd.DataFrame(client.get_historical_klines(symbol,
                                      interval,
                                      str(lookback) + ' hour ago UTC'))
    df = df.iloc[:,:6]
    df.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    df.insert(loc=0, column='Name', value=symbol)
    df.iloc[:,1:] = df.iloc[:,1:].astype(float)
    df.Time = pd.to_datetime(df.Time, unit = 'ms')
    # df.set_index('Time', inplace = True)
    
    return df

In [6]:
def get_ta(df):
    df['rsi'] = ta.momentum.rsi(df.Close, window = 14)
    df['macd'] = ta.trend.macd_diff(df.Close)
    return df

In [7]:
for pair in pairs:
    pair_data = get_ta(get_data(pair, Client.KLINE_INTERVAL_1HOUR, '300'))
    stream = pd.concat([stream, pair_data], ignore_index = True)

In [8]:
time_frame = stream.Time.max() - pd.Timedelta(hours=1)
time_frame

Timestamp('2022-11-11 13:00:00')

In [9]:
buy_now = stream[((stream['rsi'] <= 30) & (stream['Time'] == time_frame + pd.Timedelta(hours=1)))]
buy_lag_1 = stream[((stream['rsi'] <= 30) & (stream['Time'] == time_frame))]
buy_lag_2 = stream[((stream['rsi'] <= 30) & (stream['Time'] == time_frame - pd.Timedelta(days=1)))]

buy_tickers = buy_lag_1[buy_lag_1['Name'].isin(buy_lag_2['Name'])]
buy_triggered = buy_now[buy_now['Name'].isin(buy_tickers['Name'])]
buy_triggered

ticker = buy_triggered[buy_triggered.rsi == buy_triggered.rsi.min()]
ticker = ticker['Name'].to_string().split()[1]

full_df = stream[stream['Name'] == ticker]
full_df


,Name,Time,Open,High,Low,Close,Volume,rsi,macd
73200,VIDTUSDT,2022-10-30 03:00:00,0.4332,0.436,0.4312,0.4351,57317.0,NaN,NaN
73201,VIDTUSDT,2022-10-30 04:00:00,0.4351,0.4441,0.4331,0.4366,176698.7,NaN,NaN
73202,VIDTUSDT,2022-10-30 05:00:00,0.4367,0.4387,0.428,0.4287,319546.1,NaN,NaN
73203,VIDTUSDT,2022-10-30 06:00:00,0.4287,0.445,0.4266,0.4416,489793.5,NaN,NaN
73204,VIDTUSDT,2022-10-30 07:00:00,0.4416,0.465,0.4388,0.4435,2684382.3,NaN,NaN
...,...,...,...,...,...,...,...,...,...
73274,VIDTUSDT,2022-11-11 10:00:00,0.02694,0.02764,0.02683,0.02724,5010863.0,28.722084,0.003901
73275,VIDTUSDT,2022-11-11 11:00:00,0.02724,0.02757,0.02613,0.02643,6825824.0,27.646318,0.003619
73276,VIDTUSDT,2022-11-11 12:00:00,0.02647,0.027,0.02599,0.02657,5579996.0,28.147243,0.003366
73277,VIDTUSDT,2022-11-11 13:00:00,0.02657,0.02673,0.02561,0.0258,3292216.0,27.038473,0.003077


In [10]:
buy_macd = stream[((stream['macd'] > 0) & (stream['Time'] == time_frame + pd.Timedelta(hours=1)))]
buy_macd_lag_1 = stream[((stream['macd'] > 0) & (stream['Time'] == time_frame))]
buy_macd_lag_2 = stream[((stream['macd'] > 0) & (stream['Time'] == time_frame - pd.Timedelta(hours=1)))]
buy_macd_lag_3 = stream[((stream['macd'] <= 0) & (stream['Time'] == time_frame - pd.Timedelta(hours=2)))]

buy_macd_cross = buy_macd_lag_2[buy_macd_lag_2['Name'].isin(buy_macd_lag_3['Name'])]
buy_macd_tickers = buy_macd_lag_1[buy_macd_lag_1['Name'].isin(buy_macd_cross['Name'])]
buy_macd_triggered = buy_macd[buy_macd['Name'].isin(buy_macd_tickers['Name'])]
buy_macd_triggered

# macd_ticker = buy_macd_triggered['Name'].to_string().split()[1::2]
# macd_ticker
# macd_df = stream[stream['Name'] == macd_ticker]
# macd_df

,Name,Time,Open,High,Low,Close,Volume,rsi,macd


In [11]:
# Buy order
def strategy(pair, amount, open_position = True):
    #PLACE BUY ORDER
    entry = 83
    while open_position:
        tracking_data = get_ta(get_data(pair, Client.KLINE_INTERVAL_1HOUR, '300'))
#         entry = tracking_data.Open.iloc[-1]
        profit = round((((tracking_data.Close.iloc[-1] - entry) / entry) * 100),2)
        print("Current Closing Price is {} and RSI is {}".format(tracking_data.Close.iloc[-1], tracking_data.rsi.iloc[-1]))
        print("Profit is {} %".format(profit))
        if (tracking_data.rsi.iloc[-1] >= 35) & (tracking_data.rsi.iloc[-2] >= 37):
            order = client.create_order(symbol = pair,
                                        side = 'SELL',
                                        type = 'MARKET', 
                                        quantity = amount)
            print(order)
        # if tracking_data.Close.iloc[-1] >= entry * 1.03 or tracking_data.Close.iloc[-1] <= entry * 0.98:
            print("Sold at {}".format(tracking_data.Close.iloc[-1]))
            print("Profit is {} %".format(profit))
            open_position = False
            break
        time.sleep(30)

In [12]:
#strategy("AAVEUSDT", 0.975)